In [11]:
import numpy as np

# Number of samples and features
N = 1000
F = 10

# Generate synthetic data
data = np.random.randn(N, F)

# Generate coefficients for the linear relationship between data and regression values
true_coefficients = np.random.randn(F, 1)  # Generating random coefficients

# Generate regression values using the linear relationship: regression_values = data * coefficients + noise
noise = np.random.randn(N, 1) * 10  # Adding Gaussian noise to the output
regression_values = np.dot(data, true_coefficients) + noise

# Add a column of ones to the data matrix for the bias term
data_with_bias = np.hstack((data, np.ones((N, 1))))

# Calculate coefficients (including the bias term) using the pseudo-inverse
calculated_coefficients_with_bias = np.dot(np.linalg.pinv(data_with_bias), regression_values)

# Extract coefficients (excluding the bias term) and the bias value separately
calculated_coefficients = calculated_coefficients_with_bias[:-1]  # Extract coefficients
bias_value = calculated_coefficients_with_bias[-1]  # Extract the bias value

# Print the true coefficients, calculated coefficients, and bias value
print("True Coefficients:\n", true_coefficients)
print("\nCalculated Coefficients using Pseudo-inverse:\n", calculated_coefficients)
print("\nBias Value:\n", bias_value)


True Coefficients:
 [[ 0.44258214]
 [-0.21900019]
 [ 0.37937669]
 [-0.09577986]
 [-0.48766456]
 [-0.64479223]
 [ 1.2596062 ]
 [ 0.32055081]
 [ 1.28429229]
 [ 1.18008075]]

Calculated Coefficients using Pseudo-inverse:
 [[ 0.67562934]
 [ 0.12176695]
 [-0.17818586]
 [ 0.06077704]
 [-0.10280442]
 [ 0.31218516]
 [ 1.57302235]
 [-0.5771026 ]
 [ 1.61494604]
 [ 0.8893342 ]]

Bias Value:
 [0.21111336]


In [16]:
data.shape, regression_values.shape

((1000, 10), (1000, 1))

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

In [18]:
# Convert NumPy arrays to PyTorch tensors
data_tensor = torch.tensor(data, dtype=torch.float32)
regression_tensor = torch.tensor(regression_values, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(data_tensor, regression_tensor)

# Set batch size for training
batch_size = 32

# Create a DataLoader
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [118]:
# Define a neural network with one fully connected layer for regression
class FullyConnectedRegressor(nn.Module):
    def __init__(self, input_size, output_size):
        super(FullyConnectedRegressor, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.fc(x)
        return x

# Hyperparameters
input_size = F  # Number of features
output_size = 1  # Regression output

learning_rate = 0.001
num_epochs = 10

# Creating an instance of the regressor
regressor = FullyConnectedRegressor(input_size, output_size)
# set random seed
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)


# init weights with xavier
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
        
regressor.apply(init_weights)

# Define loss function and optimizer for regression (MSELoss for regression task)
criterion = nn.MSELoss(reduce='mean')
optimizer = optim.Adam(regressor.parameters(), lr=learning_rate)

/work/hpc/vinhpt/envs/thyroid-detection/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [119]:
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader, 0):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = regressor(inputs)

        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}] with Loss: {running_loss/ len(train_loader):.4f}")

print("Training finished")


Epoch [1/100] with Loss: 109.3699
Epoch [2/100] with Loss: 108.2045
Epoch [3/100] with Loss: 109.9667
Epoch [4/100] with Loss: 109.5642
Epoch [5/100] with Loss: 107.8697
Epoch [6/100] with Loss: 109.3603
Epoch [7/100] with Loss: 109.3431
Epoch [8/100] with Loss: 109.0389
Epoch [9/100] with Loss: 110.5721
Epoch [10/100] with Loss: 107.2759
Epoch [11/100] with Loss: 107.2781
Epoch [12/100] with Loss: 109.4448
Epoch [13/100] with Loss: 107.1666
Epoch [14/100] with Loss: 108.4692
Epoch [15/100] with Loss: 107.0181
Epoch [16/100] with Loss: 107.1074
Epoch [17/100] with Loss: 107.2736
Epoch [18/100] with Loss: 106.6659
Epoch [19/100] with Loss: 107.2884
Epoch [20/100] with Loss: 105.8163
Epoch [21/100] with Loss: 108.4758
Epoch [22/100] with Loss: 109.3411
Epoch [23/100] with Loss: 106.3535
Epoch [24/100] with Loss: 105.8666
Epoch [25/100] with Loss: 105.6219
Epoch [26/100] with Loss: 104.7178
Epoch [27/100] with Loss: 105.0928
Epoch [28/100] with Loss: 106.0195
Epoch [29/100] with Loss: 105

In [120]:
# Print MSE between prediction values and true regression values using coefficients learned by the neural network
prediction_values = regressor(data_tensor).detach().numpy()
mse = np.mean((prediction_values - regression_values) ** 2)
print("\nMSE between prediction values and true regression values using coefficients learned by the neural network:", mse)
mse = criterion(torch.tensor(prediction_values), regression_tensor).item()
print("\nMSE between prediction values and true regression values using coefficients learned by the neural network:", mse)

# Print MSE between prediction values and true regression values using calculated coefficients
prediction_values = np.dot(data_with_bias, calculated_coefficients_with_bias)
mse = np.mean((prediction_values - regression_values) ** 2)
print("\nMSE between prediction values and true regression values using calculated coefficients:", mse)


MSE between prediction values and true regression values using coefficients learned by the neural network: 102.41550450312357

MSE between prediction values and true regression values using coefficients learned by the neural network: 102.41549682617188

MSE between prediction values and true regression values using calculated coefficients: 102.04394509348968


In [121]:
print(regressor.state_dict())

OrderedDict([('fc.weight', tensor([[ 0.6554,  0.1635, -0.1381,  0.1643, -0.1126,  0.2285,  1.0940, -0.3826,
          1.4799,  0.6064]])), ('fc.bias', tensor([0.1666]))])


In [114]:
print("True Coefficients:\n", true_coefficients)
print("\nCalculated Coefficients using Pseudo-inverse:\n", calculated_coefficients)
print("\nBias Value:\n", bias_value)

True Coefficients:
 [[ 0.44258214]
 [-0.21900019]
 [ 0.37937669]
 [-0.09577986]
 [-0.48766456]
 [-0.64479223]
 [ 1.2596062 ]
 [ 0.32055081]
 [ 1.28429229]
 [ 1.18008075]]

Calculated Coefficients using Pseudo-inverse:
 [[ 0.67562934]
 [ 0.12176695]
 [-0.17818586]
 [ 0.06077704]
 [-0.10280442]
 [ 0.31218516]
 [ 1.57302235]
 [-0.5771026 ]
 [ 1.61494604]
 [ 0.8893342 ]]

Bias Value:
 [0.21111336]
